In [ ]:
#first Run the preprocessing part in TA's code

In [ ]:
#example of defining model:
def Basic_CNN(trim_time=500):#use trim to change input for different sequence length
  basic_cnn_model = Sequential()

# Conv block 1 conv+maxpooling+batchnorm 
  basic_cnn_model.add(Conv2D(filters=10, kernel_size=(10,1), padding='same', activation='elu', input_shape=(int(trim_time/2),1,22)))
  basic_cnn_model.add(MaxPooling2D(pool_size=(3,1), padding='same')) # Read the keras documentation
  basic_cnn_model.add(BatchNormalization())
  basic_cnn_model.add(Dropout(0.5))

# Conv block 2
  basic_cnn_model.add(Conv2D(filters=25, kernel_size=(10,1), padding='same', activation='elu'))
  basic_cnn_model.add(MaxPooling2D(pool_size=(3,1), padding='same'))
  basic_cnn_model.add(BatchNormalization())
  basic_cnn_model.add(Dropout(0.5))


# Conv Block 3
  basic_cnn_model.add(Conv2D(filters=50, kernel_size=(10,1), padding='same', activation='elu'))
  basic_cnn_model.add(MaxPooling2D(pool_size=(3,1), padding='same'))
  basic_cnn_model.add(BatchNormalization())
  basic_cnn_model.add(Dropout(0.5))

# Conv Block 4
  basic_cnn_model.add(Conv2D(filters=100, kernel_size=(10,1), padding='same', activation='elu'))
  basic_cnn_model.add(MaxPooling2D(pool_size=(3,1), padding='same'))
  basic_cnn_model.add(BatchNormalization())
  basic_cnn_model.add(Dropout(0.5))

 # Conv Block 5
  basic_cnn_model.add(Conv2D(filters=200, kernel_size=(10,1), padding='same', activation='elu'))
  basic_cnn_model.add(MaxPooling2D(pool_size=(3,1), padding='same'))
  basic_cnn_model.add(BatchNormalization())
  basic_cnn_model.add(Dropout(0.5))




# FC layer + Softmax activation
  basic_cnn_model.add(Flatten()) # Flattens the input
  basic_cnn_model.add(Dense(4, activation='softmax')) # Output FC layer with softmax activation


# Printing the model summary
  basic_cnn_model.summary()
  return basic_cnn_model

#basic_cnn_model.summary()

##(i) Train on subject 1.Does it help to train across all subjects?

In [ ]:
def process_data(X_train_valid,y_train_valid):   
    x_train_valid_p, y_train_valid_p = data_prep(X_train_valid, y_train_valid, 2, 2, True)
    y_train_valid_p = to_categorical(y_train_valid_p, 4)
    x_train_valid_p = x_train_valid_p.reshape(x_train_valid_p.shape[0], x_train_valid_p.shape[1], x_train_valid_p.shape[2], 1)
    x_train_valid_p = np.swapaxes(x_train_valid_p, 1,3)
    x_train_valid_p = np.swapaxes(x_train_valid_p, 1,2)
    return x_train_valid_p, y_train_valid_p

In [ ]:
#train:subject1
#test:subject 1-9
index_train=np.where(person_train_valid==0)
#index_train=person_train_valid.find(0)
#index_test=np.where_test==0)
X_train_subject1=X_train_valid[index_train[0]]
y_train_subject1=y_train_valid[index_train[0]]
X_test_subject=[]
y_test_subject=[]
X_train_subject_p,y_train_subject_p=process_data(X_train_subject1,y_train_subject1)
for i in range(9):
    index_test=np.where(person_test==i)
    X_test_subject_p,y_test_subject_p= process_data(X_test[index_test[0]],y_test_original[index_test[0]])
    X_test_subject.append(X_test_subject_p)
    y_test_subject.append(y_test_subject_p)

In [ ]:
#define hyperparameters and compile the model
learning_rate = 1e-4
epochs=100
cnn_optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
# Compiling the model
basic_cnn_model1=Basic_CNN()#change model here
basic_cnn_model1.compile(loss='categorical_crossentropy',
                 optimizer=cnn_optimizer,
                 metrics=['accuracy'])

# Training and validating the model

basic_cnn_model_results1 = basic_cnn_model1.fit(X_train_subject_p,
             y_train_subject_p,
             batch_size=64,
             epochs=epochs,verbose=True)# model train on subject 1

In [ ]:
#test on testdata
cnn_score_subject1=[]
for i in range(9): 
  cnn_score = basic_cnn_model1.evaluate(X_test_subject[i], y_test_subject[i], verbose=0)#test using data from differen subject
  cnn_score_subject1.append(cnn_score[1])
  print('Test accuracy of the basic CNN model:',cnn_score[1])


In [ ]:
#use the model in section 1 to test
cnn_score_total=[]
for i in range(9): 
  cnn_score = basic_cnn_model.evaluate(X_test_subject[i], y_test_subject[i], verbose=0)#model train on all subjects
  cnn_score_total.append(cnn_score[1])
  print('Test accuracy of the basic CNN model:',cnn_score[1])

##(ii) Evaluate classification accuracy as a function of time

In [ ]:
#data processing for segmenting sequence
#trim_time can be adjusted to acquire different time length
#250 500 750 1000

def process_data_new(X_train_valid,y_train_valid,trim_time):   
    x_train_valid_p, y_train_valid_p = data_prep(X_train_valid, y_train_valid, 2, 2, True,trim_time)
    y_train_valid_p = to_categorical(y_train_valid_p, 4)
    x_train_valid_p = x_train_valid_p.reshape(x_train_valid_p.shape[0], x_train_valid_p.shape[1], x_train_valid_p.shape[2], 1)
    x_train_valid_p = np.swapaxes(x_train_valid_p, 1,3)
    x_train_valid_p = np.swapaxes(x_train_valid_p, 1,2)
    return x_train_valid_p,y_train_valid_p 


In [ ]:
time_sequence=[250,500,750,1000]
CNN_time_accuracy=[]
for trim_time in time_sequence:
    X_train_sequence,y_train_sequence=process_data_new(X_train_valid,y_train_valid,trim_time=trim_time)
    X_test_sequence,y_test_sequence=process_data_new(X_test,y_test_original,trim_time=trim_time)
    print('shape of train sequence',np.shape(X_train_sequence))
    print('shape of train sequence label',np.shape(y_train_sequence))
    print('shape of test sequence',np.shape(X_test_sequence))
    print('shape of test sequence label',np.shape(y_test_sequence))
    learning_rate = 1e-4
    epochs=100
    cnn_optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    basic_cnn_model_new=Basic_CNN(trim_time=trim_time)
    basic_cnn_model_new.compile(loss='categorical_crossentropy',
                 optimizer=cnn_optimizer,
                 metrics=['accuracy'])
    new_cnn_model_results = basic_cnn_model_new.fit(X_train_sequence,y_train_sequence,
             batch_size=64,
             epochs=epochs,verbose=True)
    cnn_score= basic_cnn_model_new.evaluate(X_test_sequence,y_test_sequence,verbose=0)
    CNN_time_accuracy.append(cnn_score[1])
    print('Test accuracy of the basic CNN model:',cnn_score[1])

In [ ]:
CNN_time_accuracy